In [1]:
import matplotlib
import pathlib
import pickle

PICKLE_PROTOCOL = 4
MPL_VERSION = matplotlib.__version__

DEFAULT_PLOT_EXT = ["png", "pdf", "svg"]

matplotlib.rcParams["pdf.fonttype"] = 42
matplotlib.rcParams["ps.fonttype"] = 42
matplotlib.rcParams["font.sans-serif"] = ["Liberation Sans"]
matplotlib.rcParams["savefig.facecolor"] = "white"
matplotlib.rcParams["savefig.edgecolor"] = "white"
matplotlib.rcParams["savefig.bbox"] = "tight"

matplotlib.rcParams["axes.spines.top"] = False
matplotlib.rcParams["axes.spines.right"] = False
matplotlib.rcParams["axes.labelsize"] = 12
matplotlib.rcParams["legend.fontsize"] = 12
matplotlib.rcParams["xtick.labelsize"] = 8
matplotlib.rcParams["ytick.labelsize"] = 8
MPL_TEXT_SIZE = 14
MPL_TEXT_BBOX = dict(facecolor='none', edgecolor='black', pad=3.0, alpha=0.5)
MPL_PANEL_LABEL_SIZE = 20

matplotlib.rcParams["lines.markersize"] = 10

matplotlib.rcParams["boxplot.notch"] = False
matplotlib.rcParams["boxplot.patchartist"] = True
matplotlib.rcParams["boxplot.medianprops.color"] = "black"
matplotlib.rcParams["boxplot.flierprops.markerfacecolor"] = "lightgrey"

matplotlib.rcParams["scatter.edgecolors"] = "none"

# note to self:
# specify plot metadata here to embed in images
# for easier traceability
# matplotlib.backends.backend_agg.FigureCanvasAgg.print_png

GENERIC_MARKER="o"
MALE_MARKER="s"
FEMALE_MARKER="X"

PACBIO_MARKER="^"
PACBIO_COLOR = 0.65, 0., 0.4
ONT_MARKER="v"
ONT_COLOR = 0.1, 0.4, 0.5

def __show_available_fonts():
    # https://jonathansoma.com/
    # lede/data-studio/matplotlib/list-all-fonts-available-in-matplotlib-plus-samples/
    import matplotlib.font_manager
    from IPython.core.display import HTML

    def make_html(fontname):
        return "<p>{font}: <span style='font-family:{font}; font-size: 24px;'>{font}</p>".format(font=fontname)

    code = "\n".join([make_html(font) for font in sorted(set([f.name for f in matplotlib.font_manager.fontManager.ttflist]))])

    HTML("<div style='column-count: 2;'>{}</div>".format(code))
    return

import matplotlib.pyplot as plt
import matplotlib.patches
import matplotlib.lines
import matplotlib.colors


def build_patch_legend(labels_and_colors, edgecolor="white"):
    
    legend_elements = []
    for label, color in labels_and_colors:
        if not matplotlib.colors.is_color_like(color):
            raise ValueError(f"Not a color: {color}")
        new_element = matplotlib.patches.Patch(
            facecolor=color,
            edgecolor=edgecolor,
            label=label
        )
        legend_elements.append(new_element)
    return legend_elements


def get_pop_legend(edgecolor="white"):
    
    pops = set()
    for row in HGSVC_SAMPLES.itertuples():
        pops.add((row.supergroup, tupleize(row.rgb_rel)))
    legend = build_patch_legend(sorted(pops), edgecolor)
    return legend


def get_line_legend(line_specs):
    
    legend_elements = []
    for line_spec in line_specs:
#         if not matplotlib.colors.is_color_like(color):
#             raise ValueError(f"Not a color: {color}")
        new_element = matplotlib.lines.Line2D(
            [],[], **line_spec
        )
        legend_elements.append(new_element)
    return legend_elements


def get_pop_color(sample):
    color = HGSVC_SAMPLES.loc[
        HGSVC_SAMPLES["sample"] == sample,
        "rgb_rel_pop"
    ].values[0]
    return tupleize(color)


def get_super_color(sample):
    color = HGSVC_SAMPLES.loc[
        HGSVC_SAMPLES["sample"] == sample,
        "rgb_rel_super"
    ].values[0]
    return tupleize(color)


def tupleize(triple):
    color = tuple(map(float, triple.split(",")))
    if not matplotlib.colors.is_color_like(color):
        raise ValueError(f"Not a color: {color}")
    return color


def add_alpha(color_tuple, alpha):
    r,g,b = color_tuple
    assert 0 <= alpha <= 1
    return r,g,b,alpha


def save_figure(file_path, mpl_obj, dpi=None, extra_artists=None):

    file_path = pathlib.Path(file_path)
    file_path.parent.mkdir(exist_ok=True, parents=True)

    set_dpi = 75 if dpi is None else dpi
    add_artists = extra_artists
    assert add_artists is None or isinstance(add_artists, list)

    if file_path.suffix == ".pck":
        raise RuntimeError("Cannot move axes between figures - no caching to pickle dump")
        if not isinstance(mpl_obj, matplotlib.axes.Axes):
            raise ValueError(
                "Can only cache matplotlib.axes.Axes to pickle file:\n"
                f"{file_path}"
            )
    else:
        if not isinstance(mpl_obj, matplotlib.figure.Figure):
            raise ValueError(
                "Can only save matplotlib.figure.Figure to plot file:\n"
                f"{file_path}"
            )
    
    if file_path.suffix == ".png":
        plt.savefig(
            file_path,
            dpi=set_dpi,
            transparent=None,
            format="png",
            bbox_extra_artists=add_artists,
            backend="agg"
        )
    elif file_path.suffix == ".pdf":
        plt.savefig(
            file_path,
            transparent=None,
            format="pdf",
            bbox_extra_artists=add_artists,
            backend="pdf"
        )
    elif file_path.suffix == ".svg":
        plt.savefig(
            file_path,
            transparent=None,
            format="svg",
            bbox_extra_artists=add_artists,
            backend="svg"
        )
    elif file_path.suffix == ".pck":
        with open(file_path, "wb") as cache:
            pickle.dump(
                mpl_obj, cache,
                protocol=PICKLE_PROTOCOL,
                fix_imports=True
            )
    else:
        raise ValueError(f"Cannot handle output path: {file_path}")
